<a href="https://colab.research.google.com/github/mjalalimanesh/IBM-Data-Science-Foursquare-Project/blob/main/Intro_Foursquare_API_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>

An IBM Data Science Capstone Project Notebook

## Table of Contents

1.  <a href="#item1">Foursquare API Search Function</a>
2.  <a href="#item2">Explore a Given Venue</a>  
3.  <a href="#item3">Explore a User</a>  
4.  <a href="#item4">Foursquare API Explore Function</a>  
5.  <a href="#item5">Get Trending Venues</a>  


### Import necessary Libraries


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation



from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize


import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version


##### Make sure that you have created a Foursquare developer account and have your credentials handy


In [2]:
CLIENT_ID = 'RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR' # your Foursquare ID
CLIENT_SECRET = 'KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3' # your Foursquare Secret
VERSION = '20201106'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR
CLIENT_SECRET:KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3


#### Let's again assume that you are at Tajrish Square. So let's start by converting the Tajrish's address to its latitude and longitude coordinates.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.


In [5]:
address = 'Tajrish square, Tehran, Iran'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
print(location.address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

میدان تجریش, تجریش, منطقه ۱ شهر تهران, دهستان تجریش, شهرستان شمیرانات, استان تهران, ایران
35.806994700000004 51.428815219116515


## 1. Search for a specific venue category

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


#### Now, let's assume that it is lunch time,  food that is within 500 metres from the Tajrish Square.


In [3]:
search_query = 'restaurant'
radius = 500
print(search_query + ' .... OK!')

restaurant .... OK!


#### Define the corresponding URL


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR&client_secret=KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3&ll=35.806994700000004,51.428815219116515&v=20201106&query=restaurant&radius=500&limit=30'

#### Send the GET Request and examine the results


In [7]:
results = requests.get(url).json()

#### Get relevant part of JSON and transform it into a _pandas_ dataframe


In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
print('length of venues list : ', len(venues))
print('first element of list : ', venues[0])
# tranform venues into a dataframe
dataframe = json_normalize(venues)
print('dataframe shape ', dataframe.shape)

dataframe.head(3)

length of venues list :  30
first element of list :  {'id': '5bc0cb46ccad6b002ca5db2e', 'name': 'Rebelan Cafe & Restaurant | کافه و رستوران ربلان (کافه و رستوران ربلان)', 'location': {'address': 'Gholam Jafari St.', 'crossStreet': '7th Floor', 'lat': 35.80859986099495, 'lng': 51.42931103349665, 'labeledLatLngs': [{'label': 'display', 'lat': 35.80859986099495, 'lng': 51.42931103349665}], 'distance': 184, 'cc': 'IR', 'city': 'تهران', 'state': 'تهران', 'country': 'ایران', 'formattedAddress': ['Gholam Jafari St. (7th Floor)', 'تهران, تهران', 'ایران']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1604911723', 'hasPerk': False}
dataframe shape  (30, 18)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood
0,5bc0cb46ccad6b002ca5db2e,Rebelan Cafe & Restaurant | کافه و رستوران ربل...,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1604911723,False,Gholam Jafari St.,7th Floor,35.808600,51.429311,"[{'label': 'display', 'lat': 35.80859986099495...",184,IR,تهران,تهران,ایران,"[Gholam Jafari St. (7th Floor), تهران, تهران, ...",NaN,NaN
1,5c2798ecf05ca8002d832e82,Rouhi Café & Restaurant | کافه رستوران روحی (ک...,"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1604911723,False,Cinema Astara,Tajrish Sq.,35.806444,51.431249,"[{'label': 'display', 'lat': 35.80644381720913...",228,IR,تهران,تهران,ایران,"[Cinema Astara (Tajrish Sq.), تهران, تهران 009...",0098,District 1
2,52a1db5b11d2334522301d19,Baghcheh Traditional Restaurant | رستوران سنتی...,"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1604911723,False,Tajrish Grand Bazaar,Hammam Dogholoo Aly.,35.805183,51.430803,"[{'label': 'display', 'lat': 35.80518269459799...",269,IR,تهران,تهران,ایران,"[Tajrish Grand Bazaar (Hammam Dogholoo Aly.), ...",NaN,NaN


#### Define information of interest and filter dataframe


In [21]:
dataframe.loc[0, 'categories']

[{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
   'suffix': '.png'},
  'id': '4bf58dd8d48988d1c4941735',
  'name': 'Restaurant',
  'pluralName': 'Restaurants',
  'primary': True,
  'shortName': 'Restaurant'}]

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(3)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,Rebelan Cafe & Restaurant | کافه و رستوران ربل...,Restaurant,Gholam Jafari St.,7th Floor,35.808600,51.429311,"[{'label': 'display', 'lat': 35.80859986099495...",184,IR,تهران,تهران,ایران,"[Gholam Jafari St. (7th Floor), تهران, تهران, ...",NaN,NaN,5bc0cb46ccad6b002ca5db2e
1,Rouhi Café & Restaurant | کافه رستوران روحی (ک...,Persian Restaurant,Cinema Astara,Tajrish Sq.,35.806444,51.431249,"[{'label': 'display', 'lat': 35.80644381720913...",228,IR,تهران,تهران,ایران,"[Cinema Astara (Tajrish Sq.), تهران, تهران 009...",0098,District 1,5c2798ecf05ca8002d832e82
2,Baghcheh Traditional Restaurant | رستوران سنتی...,Persian Restaurant,Tajrish Grand Bazaar,Hammam Dogholoo Aly.,35.805183,51.430803,"[{'label': 'display', 'lat': 35.80518269459799...",269,IR,تهران,تهران,ایران,"[Tajrish Grand Bazaar (Hammam Dogholoo Aly.), ...",NaN,NaN,52a1db5b11d2334522301d19


#### Let's visualize the restaurants that are nearby


In [26]:
dataframe_filtered.name

0     Rebelan Cafe & Restaurant | کافه و رستوران ربل...
1     Rouhi Café & Restaurant | کافه رستوران روحی (ک...
2     Baghcheh Traditional Restaurant | رستوران سنتی...
3       Foloot Restaurant | رستوران فلوت (رستوران فلوت)
4     Buono Italian Restaurant (رستوران ایتالیایی بونو)
5                                      Valak Restaurant
6     Part Café & Restaurant | کافه رستوران پارت (کا...
7     Bam-e Sabz Café & Restaurant (کافه رستوران بام...
8     Hafezieh Restaurant | رستوران حافظیه (رستوران ...
9     Part cafe & Restaurant & Hooka Lounge / كافه ر...
10                 Atorina Restaurant | رستوران آتورینا
11    Le Chatean French Restaurant (رستوران فرانسوی ...
12                         Tajrasht Café And Restaurant
13                   Dalloon Restaurant | رستوران دالون
14    Restaurant Gallery Bame Shemroon‎ | رستوران گا...
15                             Hoodad cafe & restaurant
16                      رستوران دنیز | Deniz Restaurant
17                                  Bazarche Res

In [29]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Tajrish square

# add a red circle marker to represent the Tajrish square
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>


## 2. Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Let's explore a restaurant


In [31]:
venue_id = '5bc0cb46ccad6b002ca5db2e' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5bc0cb46ccad6b002ca5db2e?client_id=RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR&client_secret=KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3&v=20201106'

#### Send GET request for result


In [32]:
result = requests.get(url).json()
print(result['response']['venue'].keys())

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


### B. Get the venue's overall rating


In [33]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.6


### C. Get the number of tips


In [34]:
result['response']['venue']['tips']['count']

62

### D. Get the venue's tips

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**


#### Create URL and send GET request. Make sure to set limit to get all tips


In [48]:
## Ecco Tips
limit = 5 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()

#### Get tips and list of associated features


In [49]:
tips = results['response']['tips']['items']
print(len(tips))
tip = results['response']['tips']['items'][0]
tip.keys()

1


dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips


In [52]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,text,agreeCount,disagreeCount,id,user.firstName,user.id
0,جای رستوران الیزه باز شده. فضاش فوق العادست و یه اتاق آینه تماشایی هم داره. مشخصا خیلی هزینه دکوراسیون اینجا و سقف های متحرکش شده و در نتیجه قیمت ها بالاست ولی حتما ارزش یک بار رفتنو داره,12,0,5c3a083d4c9be6002c0d6fb0,Alireza,77983309


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all tips.


<a id="item3"></a>


## 3. Search a Foursquare User

> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### Define URL, send GET request and display features associated with user


In [95]:
idnumber = '77983309' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['tips']['count']
#user_data=results['response']['tips']['items'][0]['venue']['photos']['groups'][0]['items']#['items']
# display features associated with user
print('total number of user tips = ', user_data)


total number of user tips =  2250


well that's a lot!

### Get User's tips


In [101]:
# define tips URL
user_id='77983309'
limit = 1000
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'venue.name']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
print(tips_filtered.shape)
tips_filtered.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


(500, 5)


,text,agreeCount,disagreeCount,id,venue.name
0,طبق گفته راهنما بنا حول و روی تپه خاکی موجود ساخته شده، هیچ طبقه‌ای بر روی طبقه پایین‌تر قرار نگرفته و اتاق ها تنها در طبقه همکف واقع هستند. زیگورات فاقد هرگونه اتاق مخفی یا چیز شبیه آن در داخل است,6,0,5e44143731456200066b86c7,Chogha Zanbil Ziggurat | زیگورات چغازنبیل (زیگورات چغازنبیل)
1,تا قبل از اکتشاف، زیگورات به صورت کامل در زیر خاک مدفون بوده. تمام آجرچینی‌های مرتب مربوط به بعد از اکتشاف توسط رومن گیرشمن باستان‌شناس فرانسوی است و دیوارهای بازسازی نشده در طبقات بالا قابل تمایز است,3,0,5e441223bb09a70006e54d6b,Chogha Zanbil Ziggurat | زیگورات چغازنبیل (زیگورات چغازنبیل)
2,جکی فرنچ فرایز خوب نپخته بود و نه خودش و نه سسش جالب نبود. گارلیک چیزشف خوب ولی با سس گوجه کارخانه‌ای. جای نشستن در پیاده رو است و بیشتر به درد گرفتن و بردن می خورد. سرویس دهی ضعیف,4,0,5e3f815a60dabe00068a9f05,Pelak Restaurant | رستوران پلاک (رستوران پلاک)
3,شوخی غذای سالم را نادیده بگیرید و به چشم یک فست فود کمی متفاوت بهش نگاه کنید. منویشان تصویری است با توضیحات کافی ولی بدون قیمت که مجبورتان می کند یکی یکی قیمت‌ها را بپرسید.,4,0,5e3f7ea8314562000665afe3,Pelak Restaurant | رستوران پلاک (رستوران پلاک)
4,معمار این اثر متفاوت و زیبا رضا دانشمیر است. البته با تبدیل اخیر تمام فضاهای خالی مجموعه به سالن جدید، طراحی اولیه او را عوض کرده اند. آواسنتر اقدسیه و مسجد ناتمام مجاور تئاتر شهر از دیگر آثار اوست.,3,0,5e3edfc73145620006bf0c57,Mellat Cinema Complex | پردیس سینما گالری ملت (پردیس سینما گالری ملت)


#### Let's get the venue for the tip with the greatest number of agree counts


In [112]:
tips_sorted = tips_filtered.sort_values(by='agreeCount', ascending=False)
tips_sorted.reset_index(inplace=True)
tips_sorted.head(3)

,index,text,agreeCount,disagreeCount,id,venue.name
0,426,"A twenty years old blvd dripping with restaurants, fast foods and ice-cream shops that attracts food lovers from all over Tehran and also rich kids who want to boast their luxury cars.",29,1,5ad481833c858d2f75d0dcfe,Andarzgou Boulevard | بلوار اندرزگو (بلوار اندرزگو)
1,439,There used to be a garden here. All the trees were dried to build this vast mall. It's a sad story that tugs at your heartstrings. Not many objected in fear of the organization that was behind it.,27,0,5a449a6c838e5971b3a3cf96,Tajrish Arg Center | ارگ تجاری تجریش (ارگ تجاری تجریش)
2,456,این اپ واسه اینه که دیگران در مورد رستوران شما نظر بدن. اینستاگرامم برای شماست که از خودتون تعریف کنید. اینجا رو با پستای تبلیغیتون پر نکنید.,26,1,59b0175dd41bb77d81df040b,Shine International Restaurant | رستوران بین المللی شاین (رستوران بین المللی شاین)


In [114]:
tips_sorted.loc[2, 'venue.name']

'Shine International Restaurant | رستوران بین المللی شاین (رستوران بین المللی شاین)'

## 4. Explore a location

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**


#### So, you just finished your food at restaurant, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values.


In [115]:
address = 'Tajrish square, Tehran, Iran'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
print(location.address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

میدان تجریش, تجریش, منطقه ۱ شهر تهران, دهستان تجریش, شهرستان شمیرانات, استان تهران, ایران
35.806994700000004 51.428815219116515


#### Define URL


In [117]:
LIMIT = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR&client_secret=KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3&ll=35.806994700000004,51.428815219116515&v=20201106&radius=500&limit=1000'

#### Send GET request and examine results


In [118]:
results = requests.get(url).json()
'There are {} around Tajrish.'.format(len(results['response']['groups'][0]['items']))

'There are 72 around Tajrish.'

#### Get relevant part of JSON


In [119]:
items = results['response']['groups'][0]['items']

#### Process JSON and convert it to a clean dataframe


In [120]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,neighborhood,postalCode,id
0,Tajrish Bazaar | بازار تجریش (بازار تجریش),Market,Tajrish Sq.,35.805868,51.429956,"[{'label': 'display', 'lat': 35.80586762169362, 'lng': 51.42995589642485}]",162,IR,تهران,تهران,ایران,"[Tajrish Sq., تهران, تهران, ایران]",NaN,NaN,NaN,50336c50e4b002cea0e87182
1,Astara Movie Theater | سینما آستارا (سینما آستارا),Multiplex,Tajrish,35.806421,51.431039,"[{'label': 'display', 'lat': 35.80642050992581, 'lng': 51.43103868717868}]",210,IR,تهران,تهران,ایران,"[Tajrish, تهران, تهران, ایران]",NaN,NaN,NaN,597306a046e1b64b6b5fd229
2,Tajrish Square | میدان تجریش (میدان تجریش),Plaza,NaN,35.806680,51.429233,"[{'label': 'display', 'lat': 35.80667999426187, 'lng': 51.429233045949005}]",51,IR,تهران,تهران,ایران,"[Valiasr Ave., تهران, تهران, ایران]",NaN,NaN,NaN,4f1aaf99e4b0f67a95e996dc
3,Shemroon Kabab House | کبابسرای شمرون (کبابسرای شمرون),Persian Restaurant,Tajrish Sq.,35.806959,51.429765,"[{'label': 'display', 'lat': 35.80695939250296, 'lng': 51.42976462841034}]",85,IR,تهران,تهران,ایران,"[Tajrish Sq. (North East of Tajrish Sq.), تهران, تهران, ایران]",North East of Tajrish Sq.,NaN,NaN,50939f15e4b0b6c7168dfde1
4,GEOX | ج اوکس (ج اوکس),Shoe Store,"-1st Fl., Arg Center",35.808035,51.426988,"[{'label': 'display', 'lat': 35.80803494163345, 'lng': 51.426987646228255}]",201,IR,تهران,تهران,ایران,"[-1st Fl., Arg Center, تهران, تهران, ایران]",NaN,NaN,NaN,55ddb870498ed8c46980894e
5,Lamiz Coffee | لمیز کافی (لمیز کافی),Coffee Shop,Tajrish Sq.,35.807081,51.428087,"[{'label': 'display', 'lat': 35.807081, 'lng': 51.428087}]",66,IR,تهران,تهران,ایران,"[Tajrish Sq. (Sadabad St.), تهران, تهران, ایران]",Sadabad St.,NaN,NaN,4e07737562e100e5816fec67
6,Tajrish Arg Center | ارگ تجاری تجریش (ارگ تجاری تجریش),Shopping Mall,"Sa'd Abad St., Tajrish Sq.",35.808056,51.426921,"[{'label': 'display', 'lat': 35.80805552061461, 'lng': 51.42692092610364}]",207,IR,تهران,تهران,ایران,"[Sa'd Abad St., Tajrish Sq., تهران, تهران, ایران]",NaN,NaN,NaN,52f8853411d2e3f5f4bb0b04
7,تره بار تجریش,Market,NaN,35.805842,51.428879,"[{'label': 'display', 'lat': 35.80584182318957, 'lng': 51.42887931350617}]",128,IR,NaN,NaN,ایران,[ایران],NaN,NaN,NaN,55bde22b498e244a3cf888ef
8,Victorinox | ویکتورینوکس (ویکتورینوکس),Accessories Store,"G Fl., Arg Center",35.808082,51.427024,"[{'label': 'display', 'lat': 35.80808172299587, 'lng': 51.427023531008814}]",202,IR,تهران,تهران,ایران,"[G Fl., Arg Center, تهران, تهران, ایران]",NaN,NaN,NaN,55ddbc02498ee2d1cb079041
9,Lamiz Coffee | لمیز کافی (لمیز کافی),Coffee Shop,Bagh Ferdows,35.806948,51.425511,"[{'label': 'display', 'lat': 35.806948, 'lng': 51.425511}]",298,IR,تهران,تهران,ایران,"[Bagh Ferdows (Vali-e Asr Street), تهران, تهران, ایران]",Vali-e Asr Street,NaN,NaN,5ddbb5b6c243930008e98810


#### Let's visualize these items on the map around our location


In [122]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a id="item5"></a>


## 5. Explore Trending Venues

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**


#### Now, instead of simply exploring the area around Tajrish, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Tajrish.


In [123]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa9175c61fa2753f89bdaac'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [124]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [125]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<a id="item6"></a>


### Thank you for completing this lab!


This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook modified by Nayef Abou Tayoun ([https://www.linkedin.com/in/nayefaboutayoun/](https://www.linkedin.com/in/nayefaboutayoun?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ))

**Modified by Mohamad Jalalimanesh to test with Tehran**


This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week2_LAB1).


<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
